## Reto 1: Pruebas A/B

### 1. Objetivos:
    - Practicar evaluar los resultados de una Prueba A/B
 
---
    
### 2. Desarrollo:

### a) Evaluando una prueba A/B de flujo de ventas

El archivo 'ab_test-sales_pipeline-2' tiene datos muy similares a los que vimos en el último ejemplo. Estamos evaluando dos flujos de ventas distintos: A y B. Cada vez que alguien compró el producto gracias a uno de los dos flujos, se registró como un `1` en la columna `conversion` y un `0` en la columna `no conversion`.

Evalúa la prueba A/B para saber si los resultados obtenidos son estadísticamente significativos. El Reto consiste en los siguientes pasos:
<s>
1. Primero cuantifica cuál de los flujos de ventas fue más efectivo y en qué proporción.
2. Después aplica el test de permutación:
    1. Combina los resultados de ambos grupos en un mismo conjunto de datos
    2. Revuelve los datos
    3. Usando muestreo aleatorio sin reposición, construye un nuevo grupo A del mismo tamaño que el original.
    4. El resto de los datos conforman nuestro nuevo grupo B.
    5. Cuantifica la métrica o estadística que calculaste con los grupos originales y guarda el resultado.
    6. Repite los pasos 1-5 `R` veces para obtener una distribución de la estadística de interés.
</s>
3. Grafica tus resultados en un histograma y grafica una línea vertical que marque el resultado del experimento.
4. Cuantifica el Valor P y evalúa la significación estadística con un Alfa de 0.05 (5%).
5. Comenta tus hallazgos con tus compañeros y la experta.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HesusG/da-python-procesamiento-datos/blob/master/sesion_7/reto1.ipynb)

In [4]:
def permutation_test(df):
    # Paso 1: Cuantificar la efectividad de los flujos de ventas
    total_count = df.groupby('test')[['conversion', 'no conversion']].sum()
    total_count['success rate'] = total_count['conversion'] / (total_count['conversion'] + total_count['no conversion'])
    print(total_count)

    # Calcular la diferencia en proporción
    diferencia_en_proporcion = 1 - (total_count.loc['B']['success rate'] / total_count.loc['A']['success rate'])
    print(f'El test A fue {diferencia_en_proporcion * 100:.2f}% más efectivo que el test B para generar ventas.\n')

    # Paso 2: Aplicar el test de permutación
    value_counts = df['test'].value_counts()
    conv_a = []
    no_conv_a = []
    conv_b = []
    no_conv_b = []

    for _ in range(1000):  # R=1000 iteraciones
        # Construir un nuevo grupo A mediante muestreo aleatorio sin reposición del mismo tamaño que el original
        a = df.sample(value_counts.loc['A'], replace=False)
        # Registrar conversiones y no conversiones para el nuevo grupo A
        conv_a.append(a['conversion'].sum())
        no_conv_a.append(a['no conversion'].sum())
        
        # El resto de los datos conforman nuestro nuevo grupo B
        b = df.loc[~df.index.isin(a.index)]
        # Registrar conversiones y no conversiones para el nuevo grupo B
        conv_b.append(b['conversion'].sum())
        no_conv_b.append(b['no conversion'].sum())

    perm_results = pd.DataFrame({
        'conv_a': conv_a,
        'no_conv_a': no_conv_a,
        'conv_b': conv_b,
        'no_conv_b': no_conv_b
    })

    # Calcular la tasa de conversión para cada grupo simulado
    perm_results['a_conv_rate'] = perm_results['conv_a'] / (perm_results['conv_a'] + perm_results['no_conv_a'])
    perm_results['b_conv_rate'] = perm_results['conv_b'] / (perm_results['conv_b'] + perm_results['no_conv_b'])

    # Calcular la diferencia en la tasa de conversión entre los grupos A y B simulados
    perm_results['diff'] = 1 - (perm_results['b_conv_rate'] / perm_results['a_conv_rate'])
    return perm_results

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/beduExpert/B2-Analisis-de-Datos-con-Python-2020/master/Datasets/ab_test-sales_pipeline-2.csv')

In [5]:
permutation_test(df)

      conversion  no conversion  success rate
test                                         
A           4163          30576      0.119836
B           4841          36747      0.116404
El test A fue 2.86% más efectivo que el test B para generar ventas.



,conv_a,no_conv_a,conv_b,no_conv_b,a_conv_rate,b_conv_rate,diff
0,4065,30674,4939,36649,0.117015,0.118760,-0.014911
1,4078,30661,4926,36662,0.117390,0.118448,-0.009012
2,4106,30633,4898,36690,0.118196,0.117774,0.003565
3,4080,30659,4924,36664,0.117447,0.118400,-0.008108
4,4059,30680,4945,36643,0.116843,0.118904,-0.017646
...,...,...,...,...,...,...,...
995,4008,30731,4996,36592,0.115375,0.120131,-0.041224
996,4085,30654,4919,36669,0.117591,0.118279,-0.005852
997,4134,30605,4870,36718,0.119002,0.117101,0.015971
998,4157,30582,4847,36741,0.119664,0.116548,0.026037
